In [145]:
import pandas as pd
import random
import string

# Arabic name map (nom/prenom => Arabic)
ARABIC_NAMES = {
    "Mohamed": "محمد",
    "Sara": "سارة",
    "Youssef": "يوسف",
    "Imane": "إيمان",
    "Aya": "آية",
    "Omar": "عمر",

    "El Yamani": "اليمني",
    "Bouchraoui": "بوشراوي",
    "Khalladi": "خلادي",
    "Lamrani": "المراني",
    "Ait Taleb": "آيت طالب",
    "Bennani": "بناني"
}


def generate_cne():
    return random.choice(string.ascii_uppercase) + ''.join(random.choices(string.digits, k=9))


def generate_cni():
    prefix = ''.join(random.choices(
        string.ascii_uppercase, k=random.choice([1, 2])))
    digits = ''.join(random.choices(string.digits, k=7 - len(prefix)))
    return prefix + digits


def generate_name():
    first_names = ["Mohamed", "Sara", "Youssef", "Imane", "Aya", "Omar"]
    last_names = ["El Yamani", "Bouchraoui", "Khalladi", "Lamrani", "Ait Taleb", "Bennani"]
    last = random.choice(last_names)
    first = random.choice(first_names)
    return last.upper(), first.upper(), ARABIC_NAMES[last], ARABIC_NAMES[first]


def generate_filiere():
    return random.choice(["Informatique", "Mathématiques", "Physique", "Biologie", "Chimie", "Géologie",  "Intelligence Artificielle", "Réseaux et Télécommunications", "Systèmes Embarqués"])


def generate_csv(n=20, output="students.csv"):
    rows = []
    for i in range(n):
        year = "2024/2025"
        filiere = generate_filiere()
        nom, prenom, nom_ar, prenom_ar = generate_name()
        cne = generate_cne()
        cni = generate_cni()
        apogee = ''.join(random.choices(string.digits, k=8)).zfill(8)
        img_path = f"{cne}_{cni}.png"
        rows.append([year, filiere, nom, prenom, nom_ar,
                    prenom_ar, cne, cni, apogee, img_path])

    df = pd.DataFrame(rows, columns=[
        "year", "filiere", "nom", "prenom", "nom_ar", "prenom_ar", "cne", "cni", "apogee", "img_path"
    ])
    df.to_csv(output, index=False)
    print(f"✅ Generated {n} fake students into {output}")


# Generate 20 students
generate_csv(n=20)

✅ Generated 20 fake students into students.csv


In [147]:
from bidi.algorithm import get_display
import arabic_reshaper
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import os

# Paths
template_path = "fsbm_card_template.png"
csv_path = "students.csv"
output_folder = "output_cards"
os.makedirs(output_folder, exist_ok=True)

# Load fonts
font = ImageFont.truetype("fontsj/calibri.ttf", 22)
font_ar = font  # Or replace with Arabic-specific font if needed

fill_color = "#212120"

# Coordinates
positions = {
    "year": (110, 130),
    "filiere": (160, 195),
    "nom": (160, 230),
    "prenom": (160, 303),
    "cne": (130, 363),
    "cni": (146, 400),
    "apogee": (580, 400),
    "nom_ar": (428, 264),       # right-aligned anchor point
    "prenom_ar": (428, 330),    # right-aligned anchor point
}

# Reshape Arabic for RTL


def reshape_arabic(text):
    reshaped = arabic_reshaper.reshape(text)
    return get_display(reshaped)

# Draw Arabic right-aligned text


def draw_rtl_text(draw, position, text, font, fill):
    reshaped_text = reshape_arabic(text)
    bbox = font.getbbox(reshaped_text)  # (x0, y0, x1, y1)
    text_width = bbox[2] - bbox[0]
    rtl_position = (position[0] - text_width, position[1])
    draw.text(rtl_position, reshaped_text, font=font, fill=fill)



# Load data
df = pd.read_csv(csv_path)

# Loop and generate cards
for _, row in df.iterrows():
    card = Image.open(template_path).convert("RGBA")
    draw = ImageDraw.Draw(card)

    # Draw fields
    draw.text(positions["year"], row["year"], fill=fill_color, font=font)
    draw.text(positions["filiere"],
              row["filiere"].capitalize(), fill=fill_color, font=font)
    draw.text(positions["nom"], row["nom"].upper(), fill=fill_color, font=font)
    draw.text(positions["prenom"], row["prenom"].upper(),
              fill=fill_color, font=font)
    draw.text(positions["cne"], row["cne"], fill=fill_color, font=font)
    draw.text(positions["cni"], row["cni"], fill=fill_color, font=font)
    draw.text(positions["apogee"], str(
        row["apogee"]), fill=fill_color, font=font)

    # Arabic fields (right-aligned)
    draw_rtl_text(draw, positions["nom_ar"], str(
        row["nom_ar"]), font_ar, fill_color)
    draw_rtl_text(draw, positions["prenom_ar"], str(
        row["prenom_ar"]), font_ar, fill_color)

    # Save the final image
    filename = f"{row['cne']}_{row['cni']}.png"
    card.save(os.path.join(output_folder, filename))
    
    